In [1]:
import httpx
import pandas as pd
import datetime as dt
import streamlit as st
import sqlite3
import json  
import time
import darts
from darts import TimeSeries

In [2]:
# obtained from fuzzworks
invData = pd.read_csv('../invTypes.csv')

In [4]:
def getMarketPrices(inplace=False):
    print('Fetching market prices')
    r = httpx.get(
        'https://esi.evetech.net/latest/markets/prices/?datasource=tranquility',)
    if(r.status_code == 200):
        df = pd.read_json(r.text)
        df['date'] = dt.datetime.now()
        con = sqlite3.connect('../data/data.db')
        df.to_sql(con=con, name="market_data", if_exists="append")
        con.close()
        if(inplace):
            return(df)
        # cur = con.cursor()
        # cur.execute('select * from market_data')
        # results = cur.fetchall()
        # print(results)
    else:
        print(f"Request did not succeed! {r.status_code}")
        rjson = json.loads(r.text)
        if(rjson['timeout'] != None or 0):
            time.sleep((rjson['timeout']/1000))
getMarketPrices()
# adjusted_price, average_price, type_id

Fetching market prices


,adjusted_price,average_price,type_id,date
0,0.000000e+00,25771948.05,43691,2022-11-25 20:58:14.465705
1,1.011980e+06,1347033.78,32772,2022-11-25 20:58:14.465705
2,7.796540e+04,72606.19,32774,2022-11-25 20:58:14.465705
3,5.958966e+06,7696164.32,32780,2022-11-25 20:58:14.465705
4,7.481665e+00,6.09,32782,2022-11-25 20:58:14.465705
...,...,...,...,...
14029,0.000000e+00,3566967.92,46912,2022-11-25 20:58:14.465705
14030,0.000000e+00,41277741.94,46914,2022-11-25 20:58:14.465705
14031,0.000000e+00,13631.03,55645,2022-11-25 20:58:14.465705
14032,0.000000e+00,4596944.44,46916,2022-11-25 20:58:14.465705


In [ ]:
def getItemPrices(id):
    # refresh database with new data
    getMarketPrices()
    con = sqlite3.connect('../data.db')
    # df = pd.read_sql_table(table_name='market_data',con=con,parse_dates='date')
    # cur = con.cursor()
    # cur.execute("select * from market_data")
    df = pd.read_sql('select * from market_data', con, parse_dates=['date'])
    selected_df = df.loc[df['type_id'] == id]
    if(len(selected_df) != 0):
        return(selected_df)
    else:
        print('Sorry, either this is not a valid item id or something went wrong. :P')

In [ ]:
def getIDInfo(id):
    selected_row = invData.loc[invData['typeID'] == id]
    return selected_row
# getIDInfo(32772)

In [ ]:
def getIdFromName(name):
    # NOTE: This function could be improved with a better search engine
    selected_rows = invData[invData['typeName'].str.contains(name, False)]
    selected_rows = selected_rows[['typeName', 'typeID']]
    return(selected_rows)
# getIdFromName('Booster')

In [ ]:
# def getHistoricalData():
     # Jita region id: 30000142
# update to the rant below some shitter was calling invalid ID types causing server load, so they shut down the endpoint indefinitely for now, fucking dumb ass's
# This is makes me very angry, apparently the historical market data call doesn't work so I have to utilize time and bandwidth information to build my own db    